# Chapter 12 - Backtesting Trading Strategies

In this Notebook you can find additional content that did not make it to the book.

## B.1 Event-driven Backtesting with `backtrader`

This recipe below contains the same trading strategy (based on the simple moving average) as the one in the book, however, implemented using `backtrader`'s `Signal` approach.

### How to do it...

1. Import the libraries:

In [8]:
# FIX: Install the backtrader library
!pip install backtrader


In [ ]:
# FIX: Use conda-forge to install a pre-compiled version of TA-Lib, which avoids the C++ error
!conda install -c conda-forge ta-lib --yes

In [ ]:
from datetime import datetime
import backtrader as bt

2. Define a class representing the trading strategy:

In [ ]:
class SmaSignal(bt.Signal):
    params = (("period", 20), )
    
    def __init__(self):
        self.lines.signal = self.data - bt.ind.SMA(period=self.p.period)

3. Download data from Yahoo Finance:

In [ ]:
data = bt.feeds.YahooFinanceData(dataname="AAPL", 
                                 fromdate=datetime(2021, 1, 1),
                                 todate=datetime(2021, 12, 31))

4. Set up the backtest:

In [ ]:
cerebro = bt.Cerebro(stdstats = False)

cerebro.adddata(data)
cerebro.broker.setcash(1000.0)
cerebro.add_signal(bt.SIGNAL_LONG, SmaSignal)
cerebro.addobserver(bt.observers.BuySell)
cerebro.addobserver(bt.observers.Value)

5. Run the backtest:

In [ ]:
print(f"Starting Portfolio Value: {cerebro.broker.getvalue():.2f}")
cerebro.run()
print(f"Final Portfolio Value: {cerebro.broker.getvalue():.2f}")

6. Plot the results:

In [ ]:
cerebro.plot(iplot=True, volume=False)

## B.2 Loading custom data to `backtrader`

Below, you can see how to backtest the same strategy, this time using manually downloaded data from Yahoo Finance (using the `yfinance` library). After downloading the data and storing it in a `pandas` DataFrame, we feed it into the backtesting framework.

The same example can be inspected in `backtrader_strategies/sma_signal.py`.

### How to do it...

1. Import the libraries:

In [ ]:
from datetime import datetime
import backtrader as bt
import yfinance as yf

2. Define a class representing the trading strategy:

In [ ]:
class SmaSignal(bt.Signal):
    params = (("period", 20), )
    
    def __init__(self):
        self.lines.signal = self.data - bt.ind.SMA(period=self.p.period)

3. Download data from Yahoo Finance:

First, we download the adjusted prices from Yahoo Finance using the `yfinance` library:

In [ ]:
aapl_df = yf.download("AAPL", 
                      start="2021-01-01", 
                      end="2021-12-31",
                      progress=False,
                      auto_adjust=True)

aapl_df.head()

Then, we feed the DataFrame to `backtrader` using `bt.feeds.PandasData`. For more information, please refer to the [official documentation](https://www.backtrader.com/docu/pandas-datafeed/pandas-datafeed/).

In [ ]:
data = bt.feeds.PandasData(dataname=aapl_df)

4. Set up the backtest:

In [ ]:
cerebro = bt.Cerebro(stdstats = False)

cerebro.adddata(data)
cerebro.broker.setcash(1000.0)
cerebro.add_signal(bt.SIGNAL_LONG, SmaSignal)
cerebro.addobserver(bt.observers.BuySell)
cerebro.addobserver(bt.observers.Value)

5. Run the backtest:

In [ ]:
print(f"Starting Portfolio Value: {cerebro.broker.getvalue():.2f}")
cerebro.run()
print(f"Final Portfolio Value: {cerebro.broker.getvalue():.2f}")

The small difference in results can be easily caused by different approximation of the stock prices.

6. Plot the results:

In [ ]:
cerebro.plot(iplot=True, volume=False)

## B.3 Analyzing the results with `pyfolio`

### How to do it...

1. Import the libraries:

In [ ]:
from datetime import datetime
import backtrader as bt
import pyfolio as pf

2. Define a class representing the trading strategy:

In [ ]:
class SmaSignal(bt.Signal):
    params = (("period", 20), )
    
    def __init__(self):
        self.lines.signal = self.data - bt.ind.SMA(period=self.p.period)

3. Download data from Yahoo Finance:

In [ ]:
data = bt.feeds.YahooFinanceData(dataname="AAPL", 
                                 fromdate=datetime(2021, 1, 1),
                                 todate=datetime(2021, 12, 31))

4. Set up the backtest:

In [ ]:
cerebro = bt.Cerebro(stdstats = False)

cerebro.adddata(data)
cerebro.broker.setcash(1000.0)
cerebro.add_signal(bt.SIGNAL_LONG, SmaSignal)
cerebro.addobserver(bt.observers.BuySell)
cerebro.addobserver(bt.observers.Value)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

5. Run the backtest:

In [ ]:
backtest_result = cerebro.run()


6. Access the `pyfolio` analyzers:

In [ ]:
pyfolio_analyzer = backtest_result[0].analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfolio_analyzer.get_pf_items()

In [ ]:
transactions

7. Print the tear sheet:

In [ ]:
pf.create_full_tear_sheet(returns,
                          transactions=transactions,
                          positions=positions)